In this notebook, we try to reproduce the results from <i>Table 1</i> in <em>Yuan, Longhao & Zhao, Qibin. (2017). Completion of High Order Tensor Data with Missing
    Entries via Tensor-train Decomposition.</em>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Setup path

import os
import sys

# From jupyter notebook, starting from Tensor-Completion/
'''
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
'''

# From google colab
# Add parent packages to path
sys.path.append("/content/drive/My Drive/Tensor-Completion")
sys.path.append("/content/drive/My Drive/Tensor-Completion/Code")

# Add lib directory, containing the required libraries, to path
sys.path.append("/content/drive/My Drive/lib")

In [0]:
try:
  import cupy as cp
except:
  print("cupy not available")

import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorly as tl
import time

from Models.CP_WOPT import CP_WOPT_Model
from Models.TT_WOPT import TT_WOPT_Model
from metrics import RSE
from Utils import mask_tensor

In [0]:
def run_CP_experiment(data, missing_rates, rank=10, n_epochs=20000):
  errors = {}

  for missing_rate in missing_rates:
    # Mask the tensor
    data_masked = mask_tensor(data, missing_rate)

    # Train the model
    model = CP_WOPT_Model(data_masked, rank, init="normal", optimization="ncg")
    model.train(n_epochs)

    # Compute the RSE of our prediction
    prediction = model.predict(use_observed=False)
    error = RSE(data, prediction)

    errors[f"mr_{missing_rate}"] = error

  return errors

In [0]:
def run_TT_experiment(data, missing_rates, rank=20, n_epochs=20000):
  n_dims = len(data.shape)
  tt_ranks = tuple([1] + [rank] * (n_dims-1) + [1])
  errors = {}

  for missing_rate in missing_rates:
    # Special case for seven-way tensor and mr of 0.95
    if missing_rate == 0.95 and n_dims == 7:
      tt_ranks = tuple([1] + [8]*(n_dims-1) + [1])
    # Mask the tensor
    data_masked = mask_tensor(data, missing_rate)

    # Train the model
    model = TT_WOPT_Model(data_masked, tt_ranks, optimization="ncg")
    model.train(n_epochs)

    # Compute the RSE of our prediction
    prediction = model.predict(use_observed=False)
    error = RSE(data, prediction)

    errors[f"mr_{missing_rate}"] = error

  return errors

<h2> 3D Synthetic data <h2>

In [0]:
# Generate 3D synthetic data

shape = (30,30,30) # Shape of our synthetic data
tensor = cp.random.normal(size=shape) # Fake tensor to create our model
model = CP_WOPT_Model(tensor, rank=10, init="normal")

# We now use our model's randomly initialized factors to create the synthetic data
factors = list(model.factors.values())
data = tl.kruskal_tensor.kruskal_to_tensor((None, factors))

In [0]:
CP_errors = run_CP_experiment(data, (0, 0.5, 0.95), n_epochs=50000)

In [18]:
CP_errors

{'mr_0': 2.280895736728838e-08,
 'mr_0.5': 1.4900477708492387e-08,
 'mr_0.95': 8.695858156885864}

In [41]:
TT_errors = run_TT_experiment(data, (0, 0.5, 0.95))

         Current function value: 0.000000
         Iterations: 927
         Function evaluations: 1415
         Gradient evaluations: 1404
         Current function value: 0.000000
         Iterations: 1961
         Function evaluations: 3141
         Gradient evaluations: 3129
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 206
         Function evaluations: 305
         Gradient evaluations: 305


In [42]:
TT_errors

{'mr_0': 2.961727316706807e-07,
 'mr_0.5': 4.3279454260476613e-07,
 'mr_0.95': 1.542634903561453}

<h2>7D Synthetic Data</h2>

In [0]:
# Generate 7D synthetic data
shape = (4,4,4,4,4,4,4) # Shape of our synthetic data
tensor = cp.random.normal(size=shape) # Fake tensor to create our model
model = CP_WOPT_Model(tensor, rank=10, init="normal")

# We now use our model's randomly initialized factors to create the synthetic data
factors = list(model.factors.values())
data = tl.kruskal_tensor.kruskal_to_tensor((None, factors))

In [44]:
CP_errors = run_CP_experiment(data, (0, 0.5, 0.95))

         Current function value: 59.975584
         Iterations: 1048
         Function evaluations: 1928
         Gradient evaluations: 1916
         Current function value: 0.000000
         Iterations: 2506
         Function evaluations: 4351
         Gradient evaluations: 4342
         Current function value: 69.080686
         Iterations: 20000
         Function evaluations: 41153
         Gradient evaluations: 41153


In [45]:
CP_errors

{'mr_0': 0.03783751975163959,
 'mr_0.5': 2.820885811802258e-07,
 'mr_0.95': 79.11259086686938}

In [51]:
TT_errors = run_TT_experiment(data, (0, 0.5, 0.95))

         Current function value: 1348.080375
         Iterations: 20000
         Function evaluations: 30781
         Gradient evaluations: 30781
         Current function value: 1425.251147
         Iterations: 20000
         Function evaluations: 30514
         Gradient evaluations: 30514
         Current function value: 1.098712
         Iterations: 20000
         Function evaluations: 30176
         Gradient evaluations: 30176


In [52]:
TT_errors

{'mr_0': 0.247323957079468,
 'mr_0.5': 0.3966643898546619,
 'mr_0.95': 2.4233623365961443}